In [1]:
import argparse
import os
import os.path as osp
import shutil
import time
import warnings

import mmcv
import torch
from mmcv import Config
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction

from mmseg import digit_version
from mmseg.apis import multi_gpu_test, single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import setup_multi_processes

/opt/conda/envs/mm_s/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# config 파일과 work_dir, epoch(model)을 설정해주세요.
cfg = Config.fromfile('//opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/_trashsegmentation/swin/upernet_swin_large_patch4_window12_512x512_pretrain_384x384_22K_160k_ade20k.py')
cfg.work_dir = '/opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/work_dirs/upernet_swin_large_patch4_window12_512x512_pretrain_384x384_22K_160k_ade20k'
epoch = 'latest'


cfg.seed=2022
cfg.gpu_ids = [1]
cfg.model.train_cfg = None
cfg.data.test.test_mode = True

# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=4,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/conda/envs/mm_s/lib/python3.8/site-packages/mmseg/datasets/pipelines/test_time_aug.py:95: UserWarning: flip_direction has no effect when flip is set to False
  warnings.warn(
2022-12-30 10:36:35,130 - mmseg - INFO - Loaded 819 images
/opt/conda/envs/mm_s/lib/python3.8/site-packages/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/work_dirs/upernet_convnext_base_fp16_512x512_160k_ade20k/best_mIoU_iter_20000.pth


In [3]:
output = single_gpu_test(model, data_loader) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 4.8 task/s, elapsed: 172s, ETA:     0s

In [8]:
import json
import pandas as pd

# 저장 위치를 설정하세요.
save_dir = "/opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/submission/new_sub_convnext_base.csv"

with open("/opt/ml/input/data/test.json", "r") as f:
    test = json.load(f)

# sample_submisson.csv 열기
submission = pd.read_csv('/opt/ml/level2_semanticsegmentation_cv-level2-cv-01/mmsegmentation/submission/sample_submission.csv', index_col=None)

# out[::2,::2]
# for id, out in zip(test['images'], output):
#     submission = submission.append({"image_id" : id['file_name'], "PredictionString" : " ".join(map(str, out[::2,::2].flatten().tolist()))}, 
#                                    ignore_index=True)


for id, out in zip(test['images'], output):
    # maxpooling
    input_size = 512
    output_size = 256
    bin_size = input_size // output_size

    out = out.reshape((1,512,512))
    result = out.reshape((1, output_size, bin_size, 
                            output_size, bin_size)).max(4).max(2)

    submission = submission.append({"image_id" : id['file_name'], "PredictionString" : " ".join(map(str, result.flatten().tolist()))}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv(save_dir, index=False)